In [206]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import dgl
import scipy.sparse as sp

In [207]:
A = torch.tensor([
    [1, 0, 1, 1, 0],
    [0, 0, 1, 1, 1],
    [1, 1, 0, 0, 0]
])
A = torch.randn(1024, 20001)
cates_k = torch.randn(1, 20001)
# cates_k = torch.tensor([[0.5, 0.5, 0.2, 0.1, 0.5]])

In [208]:
A_fft = torch.fft.fft(A.T)
A_fft

tensor([[  0.2663+0.0000j, -15.3432+12.7212j, -23.8609-25.9909j,
          ...,  22.7781+9.6405j, -23.8609+25.9909j,
         -15.3432-12.7212j],
        [  8.5090+0.0000j,  32.1559+31.7601j,  23.3478-28.4980j,
          ...,   6.0848-26.8779j,  23.3478+28.4980j,
          32.1559-31.7601j],
        [ -6.7246+0.0000j,  22.2321-4.4798j,  44.9715-43.8457j,
          ..., -14.7881+21.8886j,  44.9715+43.8457j,
          22.2321+4.4798j],
        ...,
        [-87.6274+0.0000j, -13.4479-2.6808j, -33.3222-28.8702j,
          ...,  28.8600+26.9230j, -33.3222+28.8702j,
         -13.4479+2.6808j],
        [-22.8916+0.0000j,  -8.6354+20.4659j,  32.6027+8.9840j,
          ...,  32.6029-7.7228j,  32.6027-8.9840j,
          -8.6354-20.4659j],
        [  7.7594+0.0000j,  11.5779-10.8994j,  13.9761+6.5086j,
          ...,   7.8796-15.5036j,  13.9761-6.5086j,
          11.5779+10.8994j]])

In [209]:
A_abs, A_pha = torch.abs(A_fft), torch.angle(A_fft)
print(A_abs)
print(A_pha)

tensor([[ 0.2663, 19.9309, 35.2827,  ..., 24.7342, 35.2827, 19.9309],
        [ 8.5090, 45.1963, 36.8410,  ..., 27.5580, 36.8410, 45.1963],
        [ 6.7246, 22.6789, 62.8083,  ..., 26.4159, 62.8083, 22.6789],
        ...,
        [87.6274, 13.7125, 44.0892,  ..., 39.4683, 44.0892, 13.7125],
        [22.8916, 22.2132, 33.8178,  ..., 33.5051, 33.8178, 22.2132],
        [ 7.7594, 15.9011, 15.4173,  ..., 17.3911, 15.4173, 15.9011]])
tensor([[ 0.0000,  2.4494, -2.3135,  ...,  0.4004,  2.3135, -2.4494],
        [ 0.0000,  0.7792, -0.8844,  ..., -1.3482,  0.8844, -0.7792],
        [ 3.1416, -0.1988, -0.7727,  ...,  2.1650,  0.7727,  0.1988],
        ...,
        [ 3.1416, -2.9448, -2.4277,  ...,  0.7507,  2.4277,  2.9448],
        [ 3.1416,  1.9701,  0.2689,  ..., -0.2326, -0.2689, -1.9701],
        [ 0.0000, -0.7552,  0.4358,  ..., -1.1006, -0.4358,  0.7552]])


In [210]:
A_abs, A_pha = torch.abs(A_fft), torch.angle(A_fft)
print(A_abs)
print(A_pha)

tensor([[ 0.2663, 19.9309, 35.2827,  ..., 24.7342, 35.2827, 19.9309],
        [ 8.5090, 45.1963, 36.8410,  ..., 27.5580, 36.8410, 45.1963],
        [ 6.7246, 22.6789, 62.8083,  ..., 26.4159, 62.8083, 22.6789],
        ...,
        [87.6274, 13.7125, 44.0892,  ..., 39.4683, 44.0892, 13.7125],
        [22.8916, 22.2132, 33.8178,  ..., 33.5051, 33.8178, 22.2132],
        [ 7.7594, 15.9011, 15.4173,  ..., 17.3911, 15.4173, 15.9011]])
tensor([[ 0.0000,  2.4494, -2.3135,  ...,  0.4004,  2.3135, -2.4494],
        [ 0.0000,  0.7792, -0.8844,  ..., -1.3482,  0.8844, -0.7792],
        [ 3.1416, -0.1988, -0.7727,  ...,  2.1650,  0.7727,  0.1988],
        ...,
        [ 3.1416, -2.9448, -2.4277,  ...,  0.7507,  2.4277,  2.9448],
        [ 3.1416,  1.9701,  0.2689,  ..., -0.2326, -0.2689, -1.9701],
        [ 0.0000, -0.7552,  0.4358,  ..., -1.1006, -0.4358,  0.7552]])


In [211]:
A_abs = A_abs * cates_k.T

In [212]:
A_rec = A_abs * (torch.e ** (1j * A_pha))
A_rec

tensor([[-3.8333e-02+0.0000e+00j,  2.2083e+00-1.8309e+00j,
          3.4342e+00+3.7408e+00j,  ...,
         -3.2784e+00-1.3875e+00j,  3.4342e+00-3.7408e+00j,
          2.2083e+00+1.8309e+00j],
        [ 3.6035e+00+0.0000e+00j,  1.3618e+01+1.3450e+01j,
          9.8876e+00-1.2069e+01j,  ...,
          2.5769e+00-1.1382e+01j,  9.8876e+00+1.2069e+01j,
          1.3618e+01-1.3450e+01j],
        [ 7.0382e+00-1.0627e-06j, -2.3269e+01+4.6887e+00j,
         -4.7069e+01+4.5891e+01j,  ...,
          1.5478e+01-2.2909e+01j, -4.7069e+01-4.5891e+01j,
         -2.3269e+01-4.6887e+00j],
        ...,
        [ 6.1782e+01-9.3288e-06j,  9.4815e+00+1.8901e+00j,
          2.3494e+01+2.0355e+01j,  ...,
         -2.0348e+01-1.8982e+01j,  2.3494e+01-2.0355e+01j,
          9.4815e+00-1.8901e+00j],
        [ 3.9309e+00-5.9354e-07j,  1.4828e+00-3.5143e+00j,
         -5.5984e+00-1.5427e+00j,  ...,
         -5.5984e+00+1.3261e+00j, -5.5984e+00+1.5427e+00j,
          1.4828e+00+3.5143e+00j],
        [ 9.0768e-01+0

In [213]:
torch.fft.ifft(A_rec).real.T

tensor([[-0.1011, -0.1622, -0.5419,  ..., -0.9308, -0.2138,  0.0186],
        [ 0.0409,  0.1928, -1.2010,  ..., -0.4472,  0.1035, -0.0540],
        [-0.0145, -0.1299,  1.4997,  ...,  0.4356,  0.0755,  0.1317],
        ...,
        [-0.0780, -0.3066, -0.9264,  ..., -0.7737, -0.2040,  0.2137],
        [-0.1265,  0.3133,  0.1299,  ...,  0.2771,  0.0297,  0.0632],
        [-0.2204,  0.4327, -1.6882,  ...,  0.9153, -0.2346,  0.1552]])

In [214]:
print(A)
print(cates_k)

tensor([[ 0.7026, -0.3830,  0.5178,  ...,  1.3202,  1.2448,  0.1591],
        [-0.2842,  0.4552,  1.1475,  ...,  0.6342, -0.6030, -0.4612],
        [ 0.1004, -0.3067, -1.4329,  ..., -0.6178, -0.4397,  1.1255],
        ...,
        [ 0.5417, -0.7241,  0.8851,  ...,  1.0974,  1.1881,  1.8272],
        [ 0.8789,  0.7397, -0.1241,  ..., -0.3931, -0.1727,  0.5405],
        [ 1.5311,  1.0217,  1.6130,  ..., -1.2982,  1.3662,  1.3263]])
tensor([[-0.1439,  0.4235, -1.0466,  ..., -0.7051, -0.1717,  0.1170]])


In [219]:
(A * cates_k - torch.fft.ifft(A_rec).real.T).abs().max()

tensor(1.0490e-05)